In [65]:
import requests

import urllib.request
import json 

In [66]:
# Пример данных
data = ('ATTENTION: This is a MUST for ALL Computer Use', '00214.1367039e50dc6b7adb0f2aa8aba83216')

In [79]:
# формируем запрос
def send_json(x):
    MESSAGE, FILE_NAME = x
    #print(MESSAGE, FILE_NAME)
    body = {
        'MESSAGE': MESSAGE,
        'FILE_NAME': FILE_NAME
        }
    #myurl = 'http://948a-35-230-58-89.ngrok.io/' + '/predict'
    myurl = 'http://127.0.0.1:5000/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [80]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.30791267979456544


In [69]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse

In [70]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

X_test.fillna('', inplace=True)

In [71]:
X_test[['MESSAGE', 'FILE_NAME']]

,MESSAGE,FILE_NAME
0,"You could try HTTrack, available here: http://...",00420.f6140b71df992b02cc59548039eb05ca
1,"At 01:12 AM 8/24/02 -0700, Adam L. Beberg wrot...",00314.611159749e214b996589d557e335648e
2,Gary Lawrence Murphy wrote:\n\n>and say hello ...,00714.16c4d34ab2c9622fe82de9570946f9ef
3,\n\n> (and no it wasnt me even though the spel...,00571.8f35c46bee6d7a238eabf207a5696b0c
4,Gary Murphy:\n\n>How do you intend to take leg...,00991.80ac3c798bb00027efa6d80fdfb00d8e
...,...,...
1908,"Me and my friends have this brand new idea, a ...",00049.09e42d433e0661f264a25c7d4ed6e3ea
1909,see my first line: I READ THE LINKS. brickbat...,00900.04d3fc4b18f2def855155994cd956529
1910,"<!doctype html public ""-//w3c//dtd html 4.0 tr...",00749.9887b1d7cb21083c777b0623cfdb02af
1911,> Is there any way that the mailing list can b...,00324.39e5abfb6121670fb637bb0367feea19


In [74]:
response = send_json(X_test[['MESSAGE', 'FILE_NAME']].iloc[1,:])

In [75]:
response

0.15388391769775583

Сделаем N запросов и оценим время

In [81]:
N = 50

In [82]:
%%time
predictions = X_test[
                     ['MESSAGE', 'FILE_NAME']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

Wall time: 356 ms


In [83]:
predictions.values[:5]

array([0.03555344, 0.15388392, 0.09508508, 0.10314481, 0.06212979])

Посчитаем метрику

In [84]:
y_test.head()

,CATEGORY
0,0
1,0
2,0
3,0
4,0


In [85]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.5258616553547959, F-Score=1.000, Precision=1.000, Recall=1.000
